Overall guidelines:
- Only change the cells which say:
```# INSERT CODE/TEXT HERE```
- Except Q1, other content will be covered in class starting Jan 17.


# Q1. Probability (1)

Say we have three random variables $A$ and $B$ and $C$. Note that we’re using standard probability
theory notation where $P(A, B) = P(B, A)$, which simply means the joint probability of both A
and B occurring.

### Q1.1
(0.5 point)

Which of the following statements is always true?
1. $P(A|B) = P(B|A)$
2. $P(A, B) = P(A) + P(B)$
3. $P(A, B, C) = P(A)P(B)P(C)$
4. $P(A|B) = max(P(A),P(B))$
5. $P(A, B, C) = P(A)P(C)$
6. $P(A, B) = P(B|A)P(A)$
7. $P(A, B, C) = P(A)P(B|A)P(C|A, B)$
8. $P(A)=\sum_{b ∈ \text{domain}(B)} P(A, B=b)$
9. $P(A)=\sum_{b ∈ \text{domain}(B)} P(A|B=b)P(B=b)$
10.$\log(1/P(A)) = 1 - log P(A)$

### Q1.2
(0.5 point)

Now assume that A, B, and C are all independent of each other. Which of the above statements
are now true?

**\# INSERT TEXT HERE**

Answer the two subparts here by listing which statement (just the statement numbers) are true.

 **Q1.1 = Statements 6, 7, and 9 are always correct.**

 **Q1.2 = Statements 3 and 8 are always correct.**

# Q2. N-gram language model (3)

Create a 5-gram language model trained on the Shakespeare dataset. Store the relevant probabilities P(word|context) in python dictionaries. Do not use any smoothing or back-off (until Question 4). Pay special attention to beginning and end of sequences in the modeling process.

In [ ]:
import requests
import collections
import random
import sys
import numpy as np
from tqdm.notebook import tqdm
import math
from math import log
from math import exp
import copy

In [ ]:
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
response = requests.get(url)
response.raise_for_status() # Raise an exception for invalid HTTP status codes
text_data = response.text
len(text_data), text_data[:100]

(1115394,
 'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou')

In [ ]:
# sample
pos = random.randint(0, len(text_data) - 1000)
print(text_data[pos:pos + 100])

ow'd;' the Romans,
'This we received;' and each in either side
Give the all-hail to thee and cry 'Be


In [ ]:
# preprocessing - do not change
text_data = response.text
text_data = text_data.replace(',', ' , ').replace('.', ' . ').replace('?', ' ? ').replace('!', ' ! ')
text_data = text_data.replace('  ', ' ')
text_data = text_data.replace('\n\n', '\n').replace('\n', ' </s> <s> ')
text_data = '<s> ' + text_data + ' </s>'
len(text_data), text_data[:100]

(1451204,
 '<s> First Citizen: </s> <s> Before we proceed any further , hear me speak .  </s> <s> All: </s> <s> ')

In [ ]:
# train-test dataset split:
test_data = text_data[-10_000:]
text_data = text_data[:-10_000]
len(text_data), len(test_data)

(1441204, 10000)

In [ ]:
# split words
data = text_data.split(' ')
len(data), data[:20]

(314074,
 ['<s>',
  'First',
  'Citizen:',
  '</s>',
  '<s>',
  'Before',
  'we',
  'proceed',
  'any',
  'further',
  ',',
  'hear',
  'me',
  'speak',
  '.',
  '',
  '</s>',
  '<s>',
  'All:',
  '</s>'])

In [ ]:
# UPDATE: Removing unseen tokens from test_data
vocab = set(data)
test_data = ' '.join([_ for _ in test_data.split(' ') if _ in vocab]) # Adding unknown tokens to the test dataset from the training dataset.
print(test_data[:100])

one , or little .  </s> <s> GONZALO: </s> <s> How and lusty the grass looks ! how green !  </s> <s> 


In [ ]:
five = collections.defaultdict(lambda:0)
four = collections.defaultdict(lambda:0)
tri = collections.defaultdict(lambda:0)
bi = collections.defaultdict(lambda:0)
uni = collections.defaultdict(lambda:0)

#INSERT CODE HERE
def ngram_extract(data, n): # Arguments: data -> training dataset; n -> number of n-grams.
  n_grams = []
  dict_ngram = {}

  for iter in range(len(data) - n + 1):
    n_grams.append(tuple(data[iter:iter + n]))

  for grams in n_grams:
    dict_ngram[grams] = dict_ngram.get(grams, 0) + 1

  return dict_ngram


def probability(dict_ngram, previous_ngram, dict_probabilities, n): # Arguments: dict_ngram -> keys- n-grams and values- count; previous_ngram -> words excluding the final word of the n-gram;  dict_probabilities -> returns the probabilites (values) and n-grams (keys).
  # Unigram case:
  if n == 1:

    for ele, val in dict_ngram.items():
      denom = sum(list(dict_ngram.values()))

      if denom != 0:
          dict_probabilities[ele[0]] = val / denom
      else:
          dict_probabilities[ele[0]] = 0

  else: # Other n-gram cases:

    for ele, val in dict_ngram.items():
      count_lower = previous_ngram.get(ele[:n - 1], 0)

      if count_lower != 0:
          dict_probabilities[ele] = val / count_lower
      else:
          dict_probabilities[ele] = 0

  return dict_probabilities

In [ ]:
# Function call:
uni_gram = ngram_extract(data, 1)
bi_gram = ngram_extract(data, 2)
tri_gram = ngram_extract(data, 3)
four_gram = ngram_extract(data, 4)
five_gram = ngram_extract(data, 5)

uni = probability(uni_gram, uni_gram, uni, 1)
bi = probability(bi_gram, uni_gram, bi, 2)
tri = probability(tri_gram, bi_gram, tri, 3)
four = probability(four_gram, tri_gram, four, 4)
five = probability(five_gram, four_gram, five, 5)

In [ ]:
# Evaluation - Do not change
assert five[('Before', 'we', 'proceed', 'any', 'further')] == 1.0 # prob of last given prev 4
assert four[('<s>', 'First', 'Citizen:', '</s>')] == 1.0 # prob of last given prev 3
assert tri[('<s>', 'First', 'Citizen:')] == 0.172 # prob of last given prev 2
assert round(bi[('First', 'Citizen:')], 3) == 0.169 # prob of last given prev 1
assert round(uni[('Citizen:')], 5) == 0.00031 # prob of last

Using the dictionaries containing unigram, bigram, ... five-gram probabilities, return probability P(word|context) as a single scalar. Here context is a list of previous tokens, e.g., [`You`, `may`, `do`, `as`, `you`] and word is a single potential next word, e.g., `like`.

In [ ]:
def get_prob_from_lm(context = [], word = ""):

  if len(context) == 0: # Unigram
    grams = copy.deepcopy(context)
    grams.append(word)
    return uni[tuple(grams)[0]]

  elif len(context) == 1: # Bigram
    grams = copy.deepcopy(context)
    grams.append(word)
    return bi[tuple(grams)]

  elif len(context) == 2: # Trigram
    grams = copy.deepcopy(context)
    grams.append(word)
    return tri[tuple(grams)]

  elif len(context) == 3: # Fourgram
    grams = copy.deepcopy(context)
    grams.append(word)
    return four[tuple(grams)]

  elif len(context) == 4: # Fivegram
    grams = copy.deepcopy(context)
    grams.append(word)
    return five[tuple(grams)]

In [ ]:
get_prob_from_lm(["<s>", "First"], "Citizen:")

0.172

# Q3. Evaluate Perplexity (2.5)

Now let's evaluate the models quantitively using the intrinsic metric **perplexity**.

Recall perplexity is the inverse probability of the test text
$$\text{ppl}(w_1, \dots, w_n) = p(w_1, \dots, w_n)^{-\frac{1}{N}}$$

For an n-gram model, perplexity is computed by
$$\text{ppl}(w_1, \dots, w_n) = (\prod_i p(w_{i+n}|w_i^{i+n-1})^{-\frac{1}{N}}$$

To get rid of numerical issue, we usually compute through:
$$\text{ppl}(w_1, \dots, w_n) = \exp(-\frac{1}{N}\sum_i \log p(w_{i+n}|w_i^{i+n-1}))$$


*Input:*

+ **prob_function**: the `get_prob_from_lm` you implemented in Q2.
+ **data**: test data is a string of text containing multiple words.

*Output:*
+ the perplexity of given data under a language model defined by **prob_function**

Note that you do NOT need to optimize the language model in any way so as to minimize perplexity. Your reported perplexity will have no correlation with your score on this assignment, as far as it is implemented correctly.

In [ ]:
def compute_perplexity(prob_function = get_prob_from_lm, data = test_data):
  ...
  # Hint: something = prob_function(context, word)
  # INSERT CODE HERE
  p = 0
  list_word = data.split()
  N = len(list_word)

  for words in (range(N)):
    word = list_word[words]
    context = list_word[max(0, words - 4):words]
    p += log(prob_function(context, word))

  ppl = exp(-p / N)
  return ppl

In [ ]:
simple_data = 'Before we proceed any further , hear me speak .'
compute_perplexity(get_prob_from_lm, simple_data)

3.692430456031934

Can the perplexity of a sentence under any 5-gram model be 1? Explain.

**\# INSERT TEXT HERE**

No, the perplexity of a sentence under nay 5-gram model can not be 1. The perplexity of the model depends on the probability values. The perplexity can be equal to 1 only if the probability is 1, which means that this sentence has apperared both in the training and the testing dataset (this is a very rare case). Having said this, it is possible for common sentences like, Good morning ! Hello neighbor, etc. to appear both in the training and testing dataset. In such an instace, the perplexity could be equal to one.

In [ ]:
compute_perplexity(data = test_data)

ValueError: math domain error

**Explain** the resulting perplexity above on test data.
Conjecture (in text, not code) how it could be improved?

The perplexity for the above case could not be computed as the text does not appear in the training dataset and hence has resulted in a zero probability value. Smoothing techniques (add-1 smoothing) can abvoid this problem.





# Q4. Interpolation Smoothing (1)

Using the dictionaries containing unigram, bigram, ... five-gram probabilities, return probability P(word|context) where context is the previous string, e.g., `You may do as you` and word is a single potential next word, e.g., `like`.

Unlike in Q2, this time use interpolation smoothing as discussed in class.

In [ ]:
l1 = 0.2; l2 = 0.2; l3 = 0.2; l4 = 0.2; l5 = 0.2
# lambdas needed for Interpolation - you do NOT need to change these for the assignment
def get_prob_from_lm_with_interpolation(context = [], word = ""):
  # INSERT CODE HERE
  context_1 = []
  context_2 = context[-1:]
  context_3 = context[-2:]
  context_4 = context[-3:]
  context_5 = context[-4:]

  p = l1 * get_prob_from_lm(context_1, word) + l2 * get_prob_from_lm(context_2, word) + l3 * get_prob_from_lm(context_3, word) + l4 * get_prob_from_lm(context_4, word) + l5 * get_prob_from_lm(context_5, word)

  return p

get_prob_from_lm_with_interpolation(['Before', 'we', 'proceed', 'any'], 'further')

0.603281246928514

Now we re-evaluate perplexity with the new probability estimates (interpolated).

In [ ]:
compute_perplexity(prob_function = get_prob_from_lm_with_interpolation, data = simple_data)

6.245056223146501

In [ ]:
compute_perplexity(prob_function = get_prob_from_lm_with_interpolation, data = test_data)

213.15016423735972

# Q5. Text Generation (2.5)

Our final goal is to generate/infer/sample from the language model that we have created.
Given a prefix or prompt, the model can generate text according to estimated next word probabilities (similar to any LLM, e.g., GPT).

Complete the following function using the dictionaries already available to sample (you may use `random.choice` function) from the distribution of possible next tokens. Remember to randomly sample (from a weighted distribution over all possible next words in the vocabulary) instead of picking only the most likely next word.

Input:
- dictionaries `uni`, `bi`, `tri`, `four`, `five`
- **n_generate**: number of maximum words to infer. E.g., `2`
- **prefix**: the context on which to condition text generation. E.g., `as you`

Output: string output containing prefix and generated words separated by whitespace. E.g., `as you like it`

In [ ]:
prefix = 'as you say you' # "hear me"

def generate(prob_fn = get_prob_from_lm_with_interpolation, prefix = prefix, n_generate = 20):
  next_prediction = prefix.split()

  for _ in range(n_generate):
        predict_probability = {}
        context = next_prediction[-4:]

        for sent in uni.keys():
            p = prob_fn(context, sent)
            predict_probability[sent] = p

        next_prediction.append(random.choices(list(predict_probability.keys()), weights = list(predict_probability.values()))[0])

  return ' '.join(next_prediction)

generate()

"as you say you </s> <s> Where you'll <s> To With </s> You ELIZABETH: indeed , thou but the elements that turtle ! thank"

In [ ]:
generate(prob_fn = get_prob_from_lm)

'as you say you </s> <s> have none .  </s> <s> Young Ned , for thee , thine uncles and myself </s> <s>'

In [ ]:
generate(prob_fn = get_prob_from_lm_with_interpolation)

"as you say you ,  </s> <s> Being capable grant , of Stanley ? what have </s> <s> I'ld with thee with that"

Compare the outputs of the two language models. Why do you think a difference exists?

**\# INSERT TEXT HERE**

The output text generated by the model without the interpolation has resulted in a more meaningful sentence than the model which employs interpolation. This could be due to the fact that the probability weights are not tuned. This behavior is expected as thew perplexity of the model with interpolation is worse than the model without interpolation.

# The End
Voila! If you were able to complete the assignment, you have successfully created a language model in barebones Python! You have trained it on a text corpus and can use it to generate arbitrary text just like GPT-3!